<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/15_Custom_Training_Loops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds

# Device setting

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# Data load

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

## Define Normalize

In [4]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

## Setup for train dataset

In [6]:
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)


## Setup for test Dataset

In [7]:
ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(AUTOTUNE)


# Model

In [8]:
model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(10, activation="softmax"),
    ]
)

In [9]:
num_epochs = 5
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
acc_metric = keras.metrics.SparseCategoricalAccuracy()

# Training Loop

In [ ]:
for epoch in range(num_epochs):
  print(f'\nStart of Training Epoch {epoch}')
  for batch_idx, (x_batch, y_batch) in enumerate(ds_train):
    with tf.GradientTape() as tape:
      y_pred = model(x_batch, training=True)
      loss = loss_fn(y_batch, y_pred)
    
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    acc_metric.update_state(y_batch, y_pred)

  train_acc = acc_metric.result()
  print(f'Accuracy over epoch {train_acc}')
  acc_metric.reset_states()


Start of Training Epoch 0


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Accuracy over epoch 0.9287999868392944

Start of Training Epoch 1


# Test Loop

In [ ]:
for batch_idx, (x_batch, y_batch) in enumerate(ds_test):
  y_pred = model(x_batch, training=False)
  acc_metric.update_state(y_batch, y_pred)

train_acc = acc_metric.result()
print(f'Accuracy over Test Set : {train_acc}')
acc_metric.reset_states()